In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import plotly.graph_objects as go

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

path = r'D:\UniFRK\SEXTANT_sept2018\S-TFMoX\enant=S_hel=+1_KE=3.0eV' # use your path
all_files = glob.glob(path + "/*.dat")

li = []
colnames=["phi","cos(theta)","value"] 

# how to load multiple files http://jonathansoma.com/lede/foundations-2017/classes/working-with-many-files/class/
for filename in all_files:
    df = pd.read_csv(filename, delimiter=r"\s+", names=colnames, header=None) 
    # or delim_whitespace=True, it is faste
    # r"\s+" is a regex (regular expression)
    df["filename"]=filename.split("\\")[-1].split(".")[0] # adding a column with the file name
    df["filename"]=df["filename"].str.replace(" ","") # corrects for spaces
    df["filename"]=df["filename"].astype("category")
    li.append(df) # a unique DataFrame

frame = pd.concat(li, axis=0)
frame_srt=frame.groupby("filename")
frame.loc[frame["filename"] == "1_1"] # first way of selecting the right file
frame_set = frame.set_index("filename") # build a multiindex using the categories of filename
phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy() # cos(theta)
counts = frame.loc[frame["filename"] == "1_1"].iloc[:,2].to_numpy() # counts

theta_rad = np.arccos(ctheta)
phi_rad = phi * np.pi/180.

# convertion to spherical coordinates 1D vectors of shape (2000,1)
x = counts * np.sin(theta_rad) * np.cos(phi_rad)
X = x.reshape(100,200)
y = counts * np.sin(theta_rad) * np.sin(phi_rad)
Y = y.reshape(100,200)
z = counts * ctheta
Z = z.reshape(100,200)

d = np.sqrt(x**2+y**2+z**2)
d_matrix = np.sqrt(X**2+Y**2+Z**2)

In [ ]:
fig = go.Figure(data=[go.Mesh3d(x=x, y=y, z=z,
                    width=500, height=500,
                    opacity=0.5,
                    color='rgba(244,22,100,0.6)'
                  )])
fig.update_layout(title='TFMeOx MFPADs theory triangulated',margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [ ]:
# Go scatters do not provide a legend
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                    mode='markers',
                    marker=dict(
                        size=5,
                        color=d,            # set color to an array/list of desired values
                        colorscale='Viridis',   # choose a colorscale
                        opacity=1.,
                        showscale=True          # to show the legend according to the color
                   )
                )])
fig.update_layout(title='TFMeOx MFPADs theory scattered',margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, surfacecolor=d_matrix)])
fig.update_layout(title='TFMeOx MFPADs theor surfy', autosize=False,
                #   width=500, height=500,
                #   margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig.show()